In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

In [2]:
# 定义 Convolution Network 模型
class Cnn(nn.Module):
    def __init__(self,num_classes):
        super(Cnn, self).__init__()
        self.layer1 = nn.Sequential(  
            nn.Conv2d(1, 12, kernel_size=2, padding=2), 
            nn.BatchNorm2d(12),  # 对这16个结果进行规范处理，
            nn.ReLU(), # 激活函数
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(12, 24, kernel_size=2, stride=1, padding=2),# 14+2*2-5+1=14  该次卷积后output_size = 14*14
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))	# 14/2=7 池化后为7*7

        self.predict = nn.Linear(216, num_classes)
 
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.predict(out)
        return out
# use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
# if use_gpu:
#     model = model.cuda()
# # 定义loss和optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import torch
import torch.nn.functional as F

dataset = pd.read_csv('/Users/elvis/ITProjects/GitHub/jupyterPY/try_vscode/raw_price_train/1_r_price_train.csv')
np_dataset = np.array(dataset)
np_dataset = np.delete(np_dataset, 0,axis=1)

def preprocess(data,m):
    '''
    data: the dataframe of stock price
    m: the length of sequence
    '''
    adj_close = dataset["Adj Close"].tolist()
    four_d_list_dataset = np_dataset.tolist()
    #
    res_X = []
    res_y = []
    # 剔除前4个元素
    for i in range(4,len(adj_close)-m):
        res_X.append([four_d_list_dataset[i:i+m]])
        res_y.append(adj_close[i+m])
    return res_X,res_y

In [4]:
X,y = preprocess(np_dataset,5)
# # print(X)
# X = np.reshape(X,(820, 5, 6, 1))

### 准备dataloader(dataset)

In [5]:
import torchvision
from torchvision import datasets, transforms,utils
from torch.utils.data import Dataset, DataLoader
class ShareDataset(torch.utils.data.Dataset):
    def __init__(self, X,y):
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        array = np.asarray(self.X[index])
        label = torch.from_numpy(np.array(self.y[index]))
        data = torch.from_numpy(array)
        return data, label
    
    def __len__(self):
        return len(self.X)

dataloader = ShareDataset(X,y)
# np.shape(np.array(dataloader.X))

In [6]:
learning_rate = 0.1
model = Cnn(1)  # 数据大小是820 5 6
model = model.double()
# cost = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters())
cost = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

In [7]:
n_epochs = 5

for epoch in range(n_epochs):
    i = 0
    running_loss = 0.0
    running_correct = 0
    print("Epoch{}/{}".format(epoch,n_epochs))
    print("-"*10)
    for data in dataloader:
        #print("train ing")
        X_train,y_train = data
        X_train,y_train = Variable(torch.unsqueeze(X_train,dim=0)),Variable(y_train)
        outputs = model(X_train)
        _,pred = torch.max(outputs.data,1)
        optimizer.zero_grad()
        if(i == 0):
            print(outputs)
            i = 2
        loss = cost(outputs,y_train.unsqueeze(0))
        
        loss.backward()
        optimizer.step()
        running_loss += loss.data
        running_correct += torch.sum(pred == y_train.data.long())
    testing_correct = 0
    for data in dataloader:
        X_test,y_test = data
        X_test,y_test = Variable(torch.unsqueeze(X_test,dim=0)),Variable(y_test)
        outputs = model(X_test)
        _,pred = torch.max(outputs,1)
        testing_correct += torch.sum(pred == y_test.data.long())
    print("Loss is :{:.4f},Train Accuracy is:{:.4f}%,Test Accuracy is:{:.4f}".format(running_loss/820,100*running_correct/820,100*testing_correct/820))
#     print("Loss is :{:.4f},Train Accuracy is:{:.4f}%,Test Accuracy is:{:.4f}".format(running_loss/len(data_train),100*running_correct/len(data_train),100*testing_correct/len(data_test)))

Epoch0/5
----------
tensor([[-0.2761]], dtype=torch.float64, grad_fn=<AddmmBackward>)
Loss is :nan,Train Accuracy is:0.0000%,Test Accuracy is:0.0000
Epoch1/5
----------
tensor([[nan]], dtype=torch.float64, grad_fn=<AddmmBackward>)
Loss is :nan,Train Accuracy is:0.0000%,Test Accuracy is:0.0000
Epoch2/5
----------
tensor([[nan]], dtype=torch.float64, grad_fn=<AddmmBackward>)
Loss is :nan,Train Accuracy is:0.0000%,Test Accuracy is:0.0000
Epoch3/5
----------
tensor([[nan]], dtype=torch.float64, grad_fn=<AddmmBackward>)
Loss is :nan,Train Accuracy is:0.0000%,Test Accuracy is:0.0000
Epoch4/5
----------
tensor([[nan]], dtype=torch.float64, grad_fn=<AddmmBackward>)
Loss is :nan,Train Accuracy is:0.0000%,Test Accuracy is:0.0000
